In [1]:
import numpy as np
from tnestmodel.datasets import datasets
from tnestmodel.temp_wl import compute_d_rounds
from tnestmodel.temp_fast_graph import SparseTempFastGraph
from nestmodel.graph_properties import number_of_flips_possible
from nestmodel.fast_graph import FastGraph
import matplotlib.pyplot as plt

In [2]:
datasets_by_name = {dataset.name : dataset for dataset in datasets}

In [3]:
datasets_by_name

{'opsahl': <tnestmodel.datasets.CSVDataset at 0x20fbe1ea0b0>,
 'email-eu2': <tnestmodel.datasets.CSVDataset at 0x20fbe1ea0e0>,
 'email-eu3': <tnestmodel.datasets.CSVDataset at 0x20fbe1ea140>,
 'dnc': <tnestmodel.datasets.CSVDataset at 0x20fbe1ea1d0>,
 'highschool_2011': <tnestmodel.datasets.CSVDataset at 0x20fbe1ea290>,
 'hospital_ward': <tnestmodel.datasets.CSVDataset at 0x20fbe1ea380>,
 'ht09': <tnestmodel.datasets.CSVDataset at 0x20fbe1ea2c0>,
 'workplace_2013': <tnestmodel.datasets.CSVDataset at 0x20fbefdebf0>}

In [4]:
curr_name = "highschool_2011"

In [5]:
my_datasets = [datasets_by_name[curr_name]]

In [6]:
#%%snakeviz --new-tab
convergence_numbers = []
hs = []
Gs = {}
for dataset in my_datasets:
    print()
    print(dataset.name, dataset.is_directed)
    E = dataset.read_edges()
    G = SparseTempFastGraph.from_temporal_edges(E, dataset.is_directed)
    Gs[dataset.name] = G
    total_time_diff = np.max(E[:,2])-np.min(E[:,2])
    for f in [0, 0.2, 0.4, 0.6, 0.8, 1.0]:
        
        if f>0:
            hs.appen(h)
            h = int(round(total_time_diff*f))
            print(h)
            s=G.get_temporal_wl_struct(h=h, d=-1)
            convergence_numbers.append((dataset.name, f, len(s.colors_per_round)))
        else:
            hs.append(0)
            its = []
            for g in G.slices:
                #g = g.switch_directions()
                g.ensure_edges_prepared()
                g.calc_wl()
                its.append(len(g.base_partitions))
            print("amax", np.max(its), np.argmax(its), G.times[np.argmax(its)], G.times[np.argmax(its)-1], G.times[np.argmax(its)+1])
            convergence_numbers.append((dataset.name, f, np.max(its)))


highschool_2011 False
amax 5 39 54900 54880 54920
54466
108932
163398
217864
272330


In [7]:
dataset = datasets_by_name[curr_name]
E_temp = dataset.read_edges()
G = SparseTempFastGraph.from_temporal_edges(E, dataset.is_directed)
G.num_nodes

126

In [8]:
E_temp.shape

(28539, 3)

In [9]:
2*E_temp.shape[0]

57078

In [10]:
from tnestmodel.temp_utils import temp_undir_to_directed
from tnestmodel.causal_completion import get_potentially_active_nodes, remove_duplicates, collect_out_edges_per_node, _create_sparse_causal_graph

In [11]:
def my_create_sparse_causal_graph(E_temp, h, is_directed, num_nodes, should_print=True):
    if not is_directed:
        E_temp = temp_undir_to_directed(E_temp)
    pactive_nodes = get_potentially_active_nodes(E_temp, h, num_nodes)
    if should_print:
        print(len(pactive_nodes))

    pactive_nodes = remove_duplicates(pactive_nodes)
    if should_print:
        print(pactive_nodes)
        print(len(pactive_nodes))
        print(len(set(map(tuple, pactive_nodes))))
    num_all_nodes = len(pactive_nodes)
    per_node = collect_out_edges_per_node(E_temp)
    E_out, int_to_tuple = _create_sparse_causal_graph(per_node, pactive_nodes, h, num_nodes)
    return E_out, num_all_nodes, int_to_tuple

In [12]:
from tnestmodel.causal_completion import create_sparse_causal_graph

In [13]:
np.all(np.diff(E_temp[:,2].ravel())>=0)

True

In [14]:
h=0
E_out, num_all_nodes, int_to_tuple = my_create_sparse_causal_graph(E_temp, h, dataset.is_directed, G.num_nodes)

154834
[[    25  54120]
 [    65  54120]
 [    25  54140]
 ...
 [    55 326450]
 [    87 326450]
 [   124 326450]]
63495
63495


In [15]:
def apply_unmapping(E, int_to_tuple):
    return [(int_to_tuple[u], int_to_tuple[v]) for u,v in E]

In [16]:
apply_unmapping(E_out[:20,:], int_to_tuple)

[((25, 54120), (65, 54120)),
 ((65, 54120), (25, 54120)),
 ((25, 54140), (65, 54140)),
 ((65, 54140), (25, 54140)),
 ((23, 54160), (24, 54160)),
 ((24, 54160), (23, 54160)),
 ((36, 54160), (99, 54160)),
 ((99, 54160), (36, 54160)),
 ((25, 54180), (65, 54180)),
 ((25, 54180), (56, 54180)),
 ((36, 54180), (56, 54180)),
 ((56, 54180), (25, 54180)),
 ((56, 54180), (36, 54180)),
 ((56, 54180), (65, 54180)),
 ((65, 54180), (25, 54180)),
 ((65, 54180), (99, 54180)),
 ((65, 54180), (56, 54180)),
 ((99, 54180), (65, 54180)),
 ((23, 54200), (46, 54200)),
 ((36, 54200), (65, 54200))]

In [17]:
e_set = set(map(tuple,E_out))
#for u,v in e_set:
#    assert (v,u) in e_set

In [18]:
print(f'# h={h:_}\t -> {E_out.shape[0]:_}')
# h=1_000	 ->   6_937_501
# h=10_000	 ->  42_802_516
# h=100_000	 -> 125_680_221
# h=347_500	 -> 162_677_519
# h=500_000	 -> 162_677_519

# h=0	 -> 57_078


In [19]:
h=0
E_out, num_all_nodes, int_to_tuple = create_sparse_causal_graph(E_temp, h, dataset.is_directed, G.num_nodes)
G = FastGraph(np.array(E_out, dtype=np.uint32), is_directed=True, num_nodes=num_all_nodes)
wl =G.calc_wl(algorithm=("nlogn", dict(compact=False)))

nlogn


In [20]:
#wl[2].max()

In [21]:
#len(np.unique(wl[2]))

In [22]:
print(hs)
for h in hs:
    E_out, num_all_nodes, int_to_tuple = create_sparse_causal_graph(E_temp, h, dataset.is_directed, G.num_nodes)
    G = FastGraph(np.array(E_out, dtype=np.uint32), is_directed=True, num_nodes=num_all_nodes).switch_directions()
    print("# wl", len(G.calc_wl()))
    

[0, 54466, 108932, 163398, 217864, 272330]
# wl 9
# wl 6
# wl 5
# wl 5
# wl 5
# wl 5


In [23]:
# no switch
# wl 6
# wl 3
# wl 4
# wl 4
# wl 4
# wl 4

# with switch
# wl 8
# wl 4
# wl 4
# wl 4
# wl 4
# wl 4



In [24]:
from nestmodel.load_datasets import relabel_edges